<a href="https://colab.research.google.com/github/szdziedzic/intent-classification-for-polish-language/blob/main/colab_experiments/herbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.9 MB/s eta 0:00:00


In [16]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00


In [21]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=1c958fa364e498ad05b6acd17c1520692edc5c6aa08b1fee7c60353b6f12bfed
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [13]:
from datasets import load_dataset

train_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='train')
print(train_dataset[0])

{'id': '1', 'locale': 'pl-PL', 'partition': 'train', 'scenario': 16, 'intent': 48, 'utt': 'obudź mnie o dziewiątej rano w piątek', 'annot_utt': 'obudź mnie o [time : dziewiątej rano] w [date : piątek]', 'worker_id': '9', 'slot_method': {'slot': ['time', 'date'], 'method': ['translation', 'translation']}, 'judgments': {'worker_id': ['21', '11', '5'], 'intent_score': [1, 1, 1], 'slots_score': [1, 1, 1], 'grammar_score': [4, 4, 4], 'spelling_score': [2, 2, 2], 'language_identification': ['target', 'target', 'target']}}


In [14]:
val_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='validation')
print(val_dataset[0])

{'id': '11', 'locale': 'pl-PL', 'partition': 'dev', 'scenario': 8, 'intent': 40, 'utt': 'wyłącz światła proszę', 'annot_utt': 'wyłącz światła proszę', 'worker_id': '6', 'slot_method': {'slot': [], 'method': []}, 'judgments': {'worker_id': ['2', '5'], 'intent_score': [1, 1], 'slots_score': [1, 1], 'grammar_score': [4, 4], 'spelling_score': [2, 2], 'language_identification': ['target', 'target']}}


In [15]:
test_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='test')
print(test_dataset[0])

{'id': '0', 'locale': 'pl-PL', 'partition': 'test', 'scenario': 16, 'intent': 48, 'utt': 'obudź mnie o piątej rano w tym tygodniu', 'annot_utt': 'obudź mnie o [time : piątej rano] [date : w tym tygodniu]', 'worker_id': '14', 'slot_method': {'slot': ['time', 'date'], 'method': ['translation', 'translation']}, 'judgments': {'worker_id': ['5', '21', '11'], 'intent_score': [1, 1, 1], 'slots_score': [1, 1, 1], 'grammar_score': [4, 4, 4], 'spelling_score': [2, 2, 2], 'language_identification': ['target', 'target', 'target']}}


In [52]:
from transformers import AutoTokenizer, AutoModel

model_names = {
    "herbert-klej-cased-v1": {
        "tokenizer": "allegro/herbert-klej-cased-tokenizer-v1",
        "model": "allegro/herbert-klej-cased-v1",
    },
    "herbert-base-cased": {
        "tokenizer": "allegro/herbert-base-cased",
        "model": "allegro/herbert-base-cased",
    },
    "herbert-large-cased": {
        "tokenizer": "allegro/herbert-large-cased",
        "model": "allegro/herbert-large-cased",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_names["herbert-base-cased"]["tokenizer"])
model = AutoModel.from_pretrained(model_names["herbert-base-cased"]["model"])

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
output = model(
    **tokenizer.batch_encode_plus(
        [
            (
                "A potem szedł środkiem drogi w kurzawie, bo zamiatał nogami, ślepy dziad prowadzony przez tłustego kundla na sznurku.",
                "A potem leciał od lasu chłopak z butelką, ale ten ujrzawszy księdza przy drodze okrążył go z dala i biegł na przełaj pól do karczmy."
            )
        ],
        padding="longest",
        add_special_tokens=True,
        return_tensors="pt",
    )
)

In [40]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0156, -0.0107,  0.0548,  ..., -0.2931,  0.0463, -0.4071],
         [-0.1879, -0.2544, -0.1516,  ..., -0.2208,  0.2126,  0.2200],
         [-0.1579, -0.0876, -0.4259,  ...,  0.0314,  0.6145, -0.2184],
         ...,
         [ 0.0454, -0.2671, -0.1461,  ...,  0.3076,  0.8337,  0.3293],
         [ 0.0229,  0.1212,  0.0126,  ..., -0.4731,  0.1688,  0.3738],
         [-0.1684, -0.0869, -0.4155,  ...,  0.0268,  0.6186, -0.2000]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.3978e-01,  7.7380e-02, -1.0388e-01,  9.9938e-01, -4.3555e-02,
          1.6402e-01,  9.9964e-01, -9.9971e-01, -9.9978e-01,  6.9485e-01,
          9.9686e-01,  8.6258e-02,  9.9989e-01, -7.4082e-01,  4.8495e-02,
          6.2281e-02, -9.9989e-01,  2.9820e-03,  3.0078e-02,  9.9358e-01,
          9.9906e-01, -9.3512e-01, -2.7758e-02, -1.6144e-01,  1.2141e-01,
         -9.3916e-01, -8.6540e-01, -9.4444e-01, -3.4983e-02, -9.946

In [55]:
import torch
import torch.nn as nn

In [56]:
MASSIVE_INTENTS = ['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock',
            'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe',
            'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up',
            'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup',
            'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query',
            'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query',
            'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim',
            'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down',
            'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff',
            'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music',
            'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set',
            'play_game', 'alarm_remove', 'lists_remove', 'transport_taxi', 'recommendation_movies',
            'iot_coffee', 'music_query', 'play_podcasts', 'lists_query']

In [57]:
linear_layer = nn.Linear(model.config.hidden_size, len(MASSIVE_INTENTS))

In [59]:
logits = linear_layer(output.pooler_output)

In [60]:
logits

tensor([[ 0.0513,  0.2590,  0.4785,  0.2153, -0.3287,  0.2298,  0.1300,  0.2410,
         -0.0821, -0.3496,  0.0079,  0.0126,  0.5295, -0.3188,  0.3458,  0.1347,
         -0.5338, -0.2103, -0.1380, -0.6404,  0.2775, -0.2658,  0.6319, -0.1121,
          0.0081,  0.5669,  0.2138, -0.4720, -0.2965, -0.1812, -0.4321, -0.1897,
         -0.0141, -0.2631,  0.5289,  0.0055,  0.1797,  0.1467,  0.1430,  0.1156,
         -0.1846,  0.7018,  0.7268,  0.3304, -0.3989,  0.4837,  0.5781, -0.6308,
         -0.0230,  0.4017, -0.4151, -0.3063, -0.3936, -0.0394, -0.1637, -0.2300,
         -0.3218,  0.3032,  0.4132,  0.2812]], grad_fn=<AddmmBackward0>)

In [61]:
probabilities = torch.softmax(logits, dim=-1)

In [62]:
probabilities

tensor([[0.0160, 0.0197, 0.0246, 0.0189, 0.0110, 0.0192, 0.0174, 0.0194, 0.0140,
         0.0107, 0.0154, 0.0154, 0.0259, 0.0111, 0.0215, 0.0174, 0.0089, 0.0123,
         0.0133, 0.0080, 0.0201, 0.0117, 0.0287, 0.0136, 0.0154, 0.0269, 0.0189,
         0.0095, 0.0113, 0.0127, 0.0099, 0.0126, 0.0150, 0.0117, 0.0259, 0.0153,
         0.0182, 0.0176, 0.0176, 0.0171, 0.0127, 0.0307, 0.0315, 0.0212, 0.0102,
         0.0247, 0.0272, 0.0081, 0.0149, 0.0228, 0.0101, 0.0112, 0.0103, 0.0146,
         0.0129, 0.0121, 0.0110, 0.0206, 0.0230, 0.0202]],
       grad_fn=<SoftmaxBackward0>)

In [63]:
predicted_intent_index = torch.argmax(probabilities, dim=-1)

In [64]:
predicted_intent_index

tensor([42])

In [67]:
predicted_intent = MASSIVE_INTENTS[predicted_intent_index]

In [68]:
predicted_intent

'transport_query'